# <big><big><font color='#5D6D7E'>VOLUME OF SHALE (VSH) ESTIMATION FROM DEN-NEU, GR, SP (ONE WELL WITH UP TO TEN ZONES)</font></big>

This file contains the steps for the estimation of three different vsh curves (with Density-Neutron crossplot, GR curve and/or SP curve).<br>
    
    
# <font color='#229954 '>INDEX</font>
#### 1. [LOAD & QC DATA TO PANDAS DATAFRAME](#LOAD)
#### 2. [VARIABLES STATEMENT](#VAR)
#### 3. [DATA VISUALIZATION (COMPOSITE LOG)](#COMBO)
#### 4. [ZONES CREATION (1 dataframe per zone)](#ZONAS)    
#### 5. [VSH ESTIMATION (DEN-NEU CROSSPLOT)](#vsh_dn)
#### 6. [VSH ESTIMATION (GR HISTOGRAM)](#vsh_gr)
#### 7. [VSH ESTIMATION (SP HISTOGRAM)](#vsh_sp)
#### 8. [VSH CURVES VISUALIZATION](#combo_zonas)
#### 9. [FINAL DATAFRAME CREATION](#df_final)
#### 10. [FINAL DATAFRAME DATA VISUALIZATION](#combo_final)<br>

In [ ]:
import pandas as pd
import numpy as np
import lasio

import matplotlib.pyplot as plt
import mplcursors
%matplotlib inline
%matplotlib notebook
from matplotlib.ticker import StrMethodFormatter
from matplotlib.ticker import MaxNLocator
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.preprocessing import MinMaxScaler

import fnmatch
import os
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

<a id='LOAD'></a>
# <big><font color='#D35400'>LOAD & QC DATA</font></big>
Some parts of this code were taken from Andy McDonald blog, for more information visit: https://www.andymcdonald.scot/

In [ ]:
las = lasio.read("15_9-23.las")

###############################
well = las.df()
well['DEPTH'] = well.index


# sort
well = well.reindex(sorted(well.columns), axis=1)

# Check curves
for count, curve in enumerate(well):
    (count)
print ('\033[1m' + f"WELL: {las.well.WELL.value}","\n" + f"Number of curves:{count+1}")
pd.set_option('display.max_columns', 500)
well

<a id='VAR'></a>
# <big><font color='#D35400'>**VARIABLES STATEMENT**</font></big>

- Variables creation for data processing and visualization purposes.(missing curves will be created with nan values, for avoiding matplotlib errors)

In [ ]:
# VARIABLES STATEMENT

bitsize = 8.75

## ax1
crv_cal = 'CALI'            # CALIPER
crv_bit = 'BS'              # BIT SIZE
crv_gr = 'GR'               # GAMMA RAY
crv_sp = 'SPO'              # SPONTANEOUS POTENTIAL

# ax2
crv_dp_res = 'RDEP'         # DEEP RESISTIVITY
crv_sh_res = 'RMED'         # SHALLOW RESISTIVITY

#ax3
crv_neu_por = 'NPHI'        # NEUTRON POROSITY
crv_den_bulk = 'RHOB'       # BULK DENSITY
crv_nmr_por = 'MPHS'        # NUCLEAR MAGNETIC RESONANCE POROSITY

#ax4
crv_con = 'M2CX'            # CONDUCTIVITY


##############################################################

if crv_cal in well.columns:
    pass
else:
    well[crv_cal]=np.nan

##############    

if crv_bit in well.columns:
    pass
else:
    well[crv_bit]=bitsize

##############    

if crv_gr in well.columns:
    pass
else:
    well[crv_gr]=np.nan
    
##############  

if crv_sp in well.columns:
    pass
else:
    well[crv_sp]=np.nan
    
##############  

if crv_dp_res in well.columns:
    pass
else:
    well[crv_dp_res]=np.nan
    
##############  

if crv_sh_res in well.columns:
    pass
else:
    well[crv_sh_res]=np.nan
    
############## 

if crv_neu_por in well.columns:
    pass
else:
    well[crv_neu_por]=np.nan

##############  

if crv_nmr_por in well.columns:
    pass
else:
    well[crv_nmr_por]=np.nan

############## 
if crv_den_bulk in well.columns:
    pass
else:
    well[crv_den_bulk]=np.nan
    
##############      
    
if crv_con in well.columns:
    pass
else:
    well[crv_con]=1000/well[crv_dp_res]    
        
################

<a id='COMBO'></a>
# <big><font color='#D35400'>DATA VISUALIZATION</font></big>
Again, some parts of this code were taken from Andy McDonald blog, for more information visit: https://www.andymcdonald.scot/

In [ ]:
top_plot= las.well.STRT.value
bot_plot= las.well.STOP.value

################################################################################
# Total amonut of labels in the axes

ax1_ticks = 6         # Conv. curves
ax2_ticks = 6         # Resistivities
ax3_ticks = 7         # Porosities 
ax4_ticks = 6         # Conductivity

################################################################################

fig, ax = plt.subplots(figsize=(20,20))#Set up the plot axes
   
ax1 = plt.subplot2grid((1,6), (0,0), rowspan=1, colspan = 1) 
ax11 = ax1.twiny()
ax12 = ax1.twiny()

ax2 = plt.subplot2grid((1,6), (0,1), rowspan=1, colspan = 1, sharey = ax1)
ax21 = ax2.twiny()

ax3 = plt.subplot2grid((1,6), (0,2), rowspan=1, colspan = 1, sharey = ax1) #
ax31 = ax3.twiny() 
ax32 = ax3.twiny() 

ax4 = plt.subplot2grid((1,6), (0,3), rowspan=1, colspan = 1, sharey = ax1) #


################################################################################

### CAL ###
cal_min = 6
cal_max = 16

ax1.plot(well[crv_cal], well.index, color = "black", linewidth = 0.5)
ax1.set_ylabel("Depth (m)")

if abs(top_plot-bot_plot) > 500:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/99)))
    
elif abs(top_plot-bot_plot) <= 500 and abs(top_plot-bot_plot) > 200:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/49)))
else:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/24)))
    
ax1.set_xlabel(f"Caliper {well[crv_cal].name}")
ax1.set_xlim(cal_min, cal_max)
ax1.xaxis.label.set_color("black")
ax1.tick_params(axis='x', colors="black")
ax1.spines["top"].set_edgecolor("black")
ax1.set_xticks(np.arange(cal_min, cal_max+1, ((cal_max-cal_min)/(ax1_ticks-1))))
ax1.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))

ax1.fill_betweenx(well.index, well[crv_bit], well[crv_cal], where=well[crv_bit]>=well[crv_cal], interpolate=True, color='brown')
ax1.fill_betweenx(well.index, well[crv_bit], well[crv_cal], where=well[crv_bit]<=well[crv_cal], interpolate=True, color='#a6ecfa')
################################################################################

### GR ###
gr_min=0
gr_max=200

ax11.plot(well[crv_gr], well.index, color = "green", linewidth = 1.5)
ax11.set_xlabel(f"Gamma Ray ({well[crv_gr].name})")
ax11.xaxis.label.set_color("green")
ax11.set_xlim(gr_min, gr_max)
ax11.tick_params(axis='x', colors="green")
ax11.spines["top"].set_position(("axes", 1.12))
ax11.spines["top"].set_visible(True)
ax11.spines["top"].set_edgecolor("green")
ax11.title.set_color('green')
ax11.set_xticks(np.arange(gr_min, gr_max+1, ((gr_max-gr_min)/(ax1_ticks-1))))
ax11.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################

### SP ###
sp_min=-80
sp_max=20

ax12.plot(well[crv_sp], well.index, color = "red", linewidth = 1.5)
ax12.set_xlabel(f"SP ({well[crv_sp].name})")
ax12.xaxis.label.set_color("red")
ax12.set_xlim(sp_min,sp_max)
ax12.tick_params(axis='x', colors="r")
ax12.spines["top"].set_position(("axes", 1.07))
ax12.spines["top"].set_visible(True)
ax12.spines["top"].set_edgecolor("r")
ax12.title.set_color('r')
ax12.set_xticks(np.arange(sp_min, sp_max+1, ((sp_max-sp_min)/(ax1_ticks-1))))
ax12.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################

### RESD (Deep Res.) ###
res_min=0.1
res_max=100

ax2.plot(well[crv_dp_res], well.index, color = "blue", linewidth = 2)
ax2.set_xlabel(f"Deep Resistivity ({well[crv_dp_res].name})")
ax2.set_xlim(res_min, res_max)
ax2.xaxis.label.set_color("blue")
ax2.tick_params(axis='x', colors="blue")
ax2.spines["top"].set_edgecolor("blue")
ax2.set_xticks(np.arange(res_min, res_max+.1, (res_max/(ax2_ticks-1))))
ax2.set_xscale("log")
ax2.grid(True, which="both", axis='x')
ax2.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
################################################################################

### RESS (Shallow Res.)###


ax21.plot(well[crv_sh_res], well.index, color = "r", linewidth = 0.5)
ax21.set_xlabel(f"Shallow Resistivity ({well[crv_sh_res].name})")
ax21.set_xlim(res_min, res_max)
ax21.xaxis.label.set_color("r")
ax21.spines["top"].set_position(("axes", 1.07))
ax21.spines["top"].set_visible(True)
ax21.tick_params(axis='x', colors="r")
ax21.spines["top"].set_edgecolor("r")
ax21.set_xticks(np.arange(res_min, res_max+.1, ((res_max-res_min)/(ax2_ticks-1))))
ax21.set_xscale("log")
ax21.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
################################################################################

### DEN BULK ###
denbulk_min = 2.65
denbulk_max = 1.65


ax3.plot(well[crv_den_bulk], well.index, color = "red", linewidth = 1.0)
ax3.set_xlabel(f"Density Bulk ({well[crv_den_bulk].name})")
ax3.set_xlim(denbulk_min,denbulk_max)
ax3.xaxis.label.set_color("red")
ax3.tick_params(axis='x', colors="r")
ax3.spines["top"].set_edgecolor("r")
ax3.set_xticks(np.arange(denbulk_max, denbulk_min+.01, ((denbulk_min-denbulk_max)/(ax3_ticks-1))))
ax3.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### NEU POR ###
neupor_min = .6
neupor_max = 0

ax31.plot(well[crv_neu_por], well.index, color = "blue", linewidth = 1.0)
ax31.set_xlabel(f"Neutron Porosity ({well[crv_neu_por].name})")
ax31.xaxis.label.set_color("b")
ax31.set_xlim(neupor_min,neupor_max)
ax31.tick_params(axis='x', colors="blue")
ax31.spines["top"].set_position(("axes", 1.07))
ax31.spines["top"].set_visible(True)
ax31.spines["top"].set_edgecolor("blue")
ax31.set_xticks(np.arange(neupor_max, neupor_min+.01, (abs(neupor_max-neupor_min)/(ax3_ticks-1))))
ax31.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### NMR POR ###
nmrtpor_min = .6
nmrtpor_max = 0

ax32.plot(well[crv_nmr_por], well.index, color = "black", linewidth = 1.0)
ax32.set_xlabel(f"NMR Total Porosity ({well[crv_nmr_por].name})")
ax32.xaxis.label.set_color("black")
ax32.set_xlim(nmrtpor_min,nmrtpor_max)
ax32.tick_params(axis='x', colors="black")
ax32.spines["top"].set_position(("axes", 1.12))
ax32.spines["top"].set_visible(True)
ax32.spines["top"].set_edgecolor("black")
ax32.set_xticks(np.arange(nmrtpor_max, nmrtpor_min+.01, (abs(nmrtpor_max-nmrtpor_min)/(ax3_ticks-1))))
ax32.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################


### CONDUCTIVITY ###
cond_min = 1000
cond_max = 0

ax4.plot(well[crv_con], well.index, color = "black", linewidth = 1.5)
ax4.set_xlabel(f"Conductivity ({well[crv_con].name})")
ax4.set_xlim(cond_min,cond_max)
ax4.xaxis.label.set_color("black")
ax4.tick_params(axis='x', colors="black")
ax4.spines["top"].set_edgecolor("black")
ax4.set_xticks(np.arange(cond_max, cond_min+1, (abs(cond_max-cond_min)/(ax4_ticks-1))))
ax4.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################


fig.suptitle(las.well.WELL.value+"\n"+"YACIMIENTO: "+las.well.FLD.value+"\n"+"COMPAÑIA: "+las.well.COMP.value,size=26,
             x=0.36,y=1)



for ax in [ax1, ax2, ax3, ax4]:
    ax.set_ylim(bot_plot, top_plot)
    ax.grid(which='major', color='lightgrey', linestyle='-')
    ax.xaxis.set_ticks_position("top")
    ax.xaxis.set_label_position("top")
    ax.spines["top"].set_position(("axes", 1.02))
 
    
for ax in [ax2, ax3, ax4]:
    plt.setp(ax.get_yticklabels(), visible = False)
  

fig.subplots_adjust(wspace = 0.2)

mplcursors.cursor(hover=True)
plt.tight_layout()
plt.show()

<a id='ZONAS'></a>
# <big><font color='#D35400'>ZONES CREATION (1 dataframe per zone)</font></big>

In [ ]:
zon1_top = 1500
zon2_top = 3150
#zon3_top = 
#zon4_top = 3200
#zon5_top = 
#zon6_top = 
#zon7_top = 
#zon8_top = 
#zon9_top = 
#zon10_top = 

if 'well_zones' in locals():
    del well_zones
else:
    well_zones = []

well_zones = []    
if 'zon1_top' in locals():
    well_zones.append(zon1_top)
    del zon1_top
else:
    pass    
if 'zon2_top' in locals():
    well_zones.append(zon2_top)
    del zon2_top
else:
    pass
if 'zon3_top' in locals():
    well_zones.append(zon3_top)
    del zon3_top
else:
    pass
if 'zon4_top' in locals():
    well_zones.append(zon4_top)
    del zon4_top
else:
    pass
if 'zon5_top' in locals():
    well_zones.append(zon5_top)
    del zon5_top
else:
    pass
if 'zon6_top' in locals():
    well_zones.append(zon6_top)
    del zon6_top
else:
    pass
if 'zon7_top' in locals():
    well_zones.append(zon7_top)
    del zon7_top
else:
    pass
if 'zon8_top' in locals():
    well_zones.append(zon7_top)
    del zon8_top
else:
    pass
if 'zon9_top' in locals():
    well_zones.append(zon7_top)
    del zon9_top
else:
    pass
if 'zon10_top' in locals():
    well_zones.append(zon7_top)
    del zon10_top
else:
    pass

#### 

well['DEPTH_int'] = well['DEPTH'].round()

###

for a,b in enumerate(well_zones):
    if a > 0:
        globals()[f"well_{well_zones[a-1]}_{well_zones[a]}"] = well[(well['DEPTH_int'] >=  well_zones[a-1]) & (well['DEPTH_int'] <=  well_zones[a])]

print ('\033[1m' + 'DATASETS CREADOS \n')
for match in fnmatch.filter(dir(), 'well_*'):
    if match == 'well_zones' or match=='well_nan' or match=='well_list':
        pass
    else:    
        print(match)

<a id='VSH'></a>
# <big><big><font color='#y67E22'>**VSH ESTIMATION**</font></big></big>


<a id='vsh_dn'></a>
# <font color='#D35400'>VSH ESTIMATION (DEN-NEU CROSSPLOT)</font>

In [ ]:
if 'df' in locals():
    df_backup_old = df
else:
    pass

################################################

well_zone_file = well_1500_3150         # RENAME WELL_ZONE FILE HERE

################################################


den_sh = 2.57
neu_sh = 0.32

den_cl = 2.6
den_fl = 1.00
neu_cl = 0.03
neu_fl = 1

###########

crv_size_min=0
crv_size_max=10

crv_x = crv_neu_por
crv_y = crv_den_bulk
crv_color= crv_gr
crv_sizeo = crv_cal
crv_size = 'SIZE_NORM'


# df 
df  = well_zone_file[well_zone_file[crv_x].notnull() & well_zone_file[crv_y].notnull()]


df[crv_size]=df[crv_sizeo].fillna(method="ffill")

df[crv_size] = (((df[crv_size]-df[crv_size
                                    ].min())/(df [crv_size].max()-df [crv_size].min()))*
              (crv_size_max-crv_size_min))+(crv_size_min)


fig = px.scatter(df , x=df [crv_x], y=df [crv_y],
                color=df [crv_color],range_color=[0,150], color_continuous_scale=px.colors.sequential.Hot_r,
                size = df [crv_size])

fig.update_layout(autosize=False, width=900, height=700) # resize figure


fig.update_coloraxes(colorbar_len=0.9)  # https://plotly.com/python/reference/layout/coloraxis/
fig.add_scatter(x=[neu_cl,neu_fl,neu_sh,neu_cl], y=[den_cl,den_fl,den_sh,den_cl])
fig.update_xaxes(range=[-0.1, .6])
fig.update_yaxes(range=[2.8, 1.7])
fig.update_layout(showlegend=False)
fig.update_layout(title=f'Crossplot Density - Neutron',title_x=0.5,
                   xaxis_title=crv_x,
                   yaxis_title=crv_y)
fig.show()

In [ ]:
# THE MATHS

neu = df[crv_neu_por]
den = df[crv_den_bulk]


m1 = ((neu_fl-neu_cl)/(den_fl-den_cl))

df['vsh_dn'] = (((neu-neu_cl)+(m1*(den_cl-den)))/((neu_sh-neu_cl)+(m1*(den_cl-den_sh))))
df['vsh_dn'] = df['vsh_dn'].clip(upper=1,lower=0.0001)
df

<a id='vsh_gr'></a>
# <font color='#D35400'>**VSH GR**</font>

**VSH_GR** = (GRlog - GRmin) / (GRmax -GRmin)

In [ ]:
fig = px.ecdf(df, x=crv_gr, marginal="histogram")
fig.update_xaxes(range=[0, 200])
fig.update_layout(showlegend=False)
fig.update_layout(title=f'Histogram {crv_gr}',title_x=0.5)
fig.show()

In [ ]:
gr_min = 20
gr_max = 120
gr_log = df[crv_gr]


df['vsh_gr'] = (gr_log - gr_min) / (gr_max - gr_min)
df['vsh_gr'] = df['vsh_gr'].clip(upper=1,lower=0.0001)
df

<a id='vsh_sp'></a>
# <font color='#D35400'>**VSH SP**</font>

**VSH_SP** = 1 - (SPlog - SPsh)/(SPcl-SPsh)

In [ ]:
fig = px.ecdf(df, x=crv_sp, marginal="histogram")
fig.update_xaxes(range=[-40, 5])
fig.update_layout(showlegend=False)
fig.update_layout(title=f'Histogram {crv_sp}',title_x=0.5)
fig.show()

In [ ]:
sp_cl = -20
sp_sh = 3
sp_log = df[crv_sp]


df['vsh_sp'] = (sp_log - sp_cl) / (sp_sh - sp_cl)
df['vsh_sp'] = df['vsh_sp'].clip(upper=1,lower=0.0001)
df

<a id='combo_zonas'></a>
# <font color='#D35400'>VSH CURVES VISUALIZATION AND well_xxxx_xxxx_final CREATION</font>

- Here, we'll be able for VSH curves visualization through a matplotlib plot.
- Also, running this cell, we are gonna create a dataframe **(well_xxxx_xxxx_final)** (the idea behind this is that, if we are pleased with the results, we will have the final zone file data already created).

In [ ]:
# well_xxxx_xxxx_final CREATION

well_final_name = [name for name in globals() if globals()[name] is well_zone_file]
globals()[f'{well_final_name[0]}_final'] = df
#####################################################

# TOP & BOTTOM 

top_plot=well_zone_file.index[0]
bot_plot=well_zone_file.index[-1]



# NUMBER OF TICKS IN THE TRACK

ax1_ticks = 6         # Convencionales 
ax2_ticks = 6         # Resistividades
ax3_ticks = 7         # Porosidades
ax4_ticks = 5         # VSH

########################################################

#  VSH CURVES

crv_vsh_dn = 'vsh_dn'
crv_vsh_gr = 'vsh_gr' 
crv_vsh_sp = 'vsh_sp'

if crv_vsh_dn in df.columns:
    pass
else:
    df[crv_vsh_dn]=np.nan
    
    
if crv_vsh_gr in df.columns:
    pass
else:
    df[crv_vsh_gr]=np.nan
    
    
if crv_vsh_sp in df.columns:
    pass
else:
    df[crv_vsh_sp]=np.nan


############################

fig, ax = plt.subplots(figsize=(20,20))#Set up the plot axes
   
ax1 = plt.subplot2grid((1,6), (0,0), rowspan=1, colspan = 1) 
ax11 = ax1.twiny()
ax12 = ax1.twiny()

ax2 = plt.subplot2grid((1,6), (0,1), rowspan=1, colspan = 1, sharey = ax1)
ax21 = ax2.twiny()

ax3 = plt.subplot2grid((1,6), (0,2), rowspan=1, colspan = 1, sharey = ax1) #
ax31 = ax3.twiny() 
ax32 = ax3.twiny() 

ax4 = plt.subplot2grid((1,6), (0,3), rowspan=1, colspan = 1, sharey = ax1)
ax41 = ax4.twiny() 
ax42 = ax4.twiny()


################################################################################

### CAL ###
cal_min = 6
cal_max = 16

ax1.plot(df[crv_cal], df.index, color = "black", linewidth = 0.5)
ax1.set_ylabel("Depth (m)")

if abs(top_plot-bot_plot) > 500:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/99)))
    
elif abs(top_plot-bot_plot) <= 500 and abs(top_plot-bot_plot) > 200:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/49)))
else:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/24)))
    
ax1.set_xlabel(f"Caliper {df[crv_cal].name}")
ax1.set_xlim(cal_min, cal_max)
ax1.xaxis.label.set_color("black")
ax1.tick_params(axis='x', colors="black")
ax1.spines["top"].set_edgecolor("black")
ax1.set_xticks(np.arange(cal_min, cal_max+1, ((cal_max-cal_min)/(ax1_ticks-1))))
ax1.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))

ax1.fill_betweenx(df.index, df[crv_bit], df[crv_cal], where=df[crv_bit]>=df[crv_cal], interpolate=True, color='brown')
ax1.fill_betweenx(df.index, df[crv_bit], df[crv_cal], where=df[crv_bit]<=df[crv_cal], interpolate=True, color='#a6ecfa')
################################################################################

### GR ###
gr_min=0
gr_max=200

ax11.plot(df[crv_gr], df.index, color = "green", linewidth = 1.5)
ax11.set_xlabel(f"Gamma Ray ({df[crv_gr].name})")
ax11.xaxis.label.set_color("green")
ax11.set_xlim(gr_min, gr_max)
ax11.tick_params(axis='x', colors="green")
ax11.spines["top"].set_position(("axes", 1.12))
ax11.spines["top"].set_visible(True)
ax11.spines["top"].set_edgecolor("green")
ax11.title.set_color('green')
ax11.set_xticks(np.arange(gr_min, gr_max+1, ((gr_max-gr_min)/(ax1_ticks-1))))
ax11.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################

### SP ###
sp_min=-80
sp_max=20

ax12.plot(df[crv_sp], df.index, color = "red", linewidth = 1.5)
ax12.set_xlabel(f"SP ({df[crv_sp].name})")
ax12.xaxis.label.set_color("red")
ax12.set_xlim(sp_min,sp_max)
ax12.tick_params(axis='x', colors="r")
ax12.spines["top"].set_position(("axes", 1.07))
ax12.spines["top"].set_visible(True)
ax12.spines["top"].set_edgecolor("r")
ax12.title.set_color('r')
ax12.set_xticks(np.arange(sp_min, sp_max+1, ((sp_max-sp_min)/(ax1_ticks-1))))
ax12.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################

### RESD ###
res_min=0.1
res_max=100

ax2.plot(df[crv_dp_res], df.index, color = "blue", linewidth = 2)
ax2.set_xlabel(f"Deep Resistivity ({df[crv_dp_res].name})")
ax2.set_xlim(res_min, res_max)
ax2.xaxis.label.set_color("blue")
ax2.tick_params(axis='x', colors="blue")
ax2.spines["top"].set_edgecolor("blue")
ax2.set_xticks(np.arange(res_min, res_max+.1, (res_max/(ax2_ticks-1))))
ax2.set_xscale("log")
ax2.grid(True, which="both", axis='x')
ax2.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
################################################################################

### RESS ###


ax21.plot(df[crv_sh_res], df.index, color = "r", linewidth = 0.5)
ax21.set_xlabel(f"Shallow Resistivity ({df[crv_sh_res].name})")
ax21.set_xlim(res_min, res_max)
ax21.xaxis.label.set_color("r")
ax21.spines["top"].set_position(("axes", 1.07))
ax21.spines["top"].set_visible(True)
ax21.tick_params(axis='x', colors="r")
ax21.spines["top"].set_edgecolor("r")
ax21.set_xticks(np.arange(res_min, res_max+.1, ((res_max-res_min)/(ax2_ticks-1))))
ax21.set_xscale("log")
ax21.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
################################################################################

### DEN BULK ###
denbulk_min = 2.65
denbulk_max = 1.65


ax3.plot(df[crv_den_bulk], df.index, color = "red", linewidth = 1.0)
ax3.set_xlabel(f"Density Bulk ({df[crv_den_bulk].name})")
ax3.set_xlim(denbulk_min,denbulk_max)
ax3.xaxis.label.set_color("red")
ax3.tick_params(axis='x', colors="r")
ax3.spines["top"].set_edgecolor("r")
ax3.set_xticks(np.arange(denbulk_max, denbulk_min+.01, ((denbulk_min-denbulk_max)/(ax3_ticks-1))))
ax3.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### NEU POR ###
neupor_min = .6
neupor_max = 0

ax31.plot(df[crv_neu_por], df.index, color = "blue", linewidth = 1.0)
ax31.set_xlabel(f"Neutron Porosity ({df[crv_neu_por].name})")
ax31.xaxis.label.set_color("b")
ax31.set_xlim(neupor_min,neupor_max)
ax31.tick_params(axis='x', colors="blue")
ax31.spines["top"].set_position(("axes", 1.07))
ax31.spines["top"].set_visible(True)
ax31.spines["top"].set_edgecolor("blue")
ax31.set_xticks(np.arange(neupor_max, neupor_min+.01, (abs(neupor_max-neupor_min)/(ax3_ticks-1))))
ax31.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### NMR POR ###
nmrtpor_min = .6
nmrtpor_max = 0

ax32.plot(df[crv_nmr_por], df.index, color = "black", linewidth = 1.0)
ax32.set_xlabel(f"NMR Total Porosity ({df[crv_nmr_por].name})")
ax32.xaxis.label.set_color("black")
ax32.set_xlim(nmrtpor_min,nmrtpor_max)
ax32.tick_params(axis='x', colors="black")
ax32.spines["top"].set_position(("axes", 1.12))
ax32.spines["top"].set_visible(True)
ax32.spines["top"].set_edgecolor("black")
ax32.set_xticks(np.arange(nmrtpor_max, nmrtpor_min+.01, (abs(nmrtpor_max-nmrtpor_min)/(ax3_ticks-1))))
ax32.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### VSH DEN-NEU ###
vsh_min = 1
vsh_max = 0

ax4.plot(df[crv_vsh_dn], df.index, color = "black", linewidth = 1)
ax4.set_xlabel(f"VSH ({crv_vsh_dn})")
ax4.set_xlim(vsh_min,vsh_max)
ax4.xaxis.label.set_color("black")
ax4.tick_params(axis='x', colors="black")
ax4.spines["top"].set_edgecolor("black")
ax4.set_xticks(np.arange(vsh_max, vsh_min+.01, ((vsh_min-vsh_max)/(ax4_ticks-1))))
ax4.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### VSH GR ###


ax41.plot(df[crv_vsh_gr], df.index, color = "green", linewidth = 1)
ax41.set_xlabel(f"VSH ({crv_vsh_gr})")
ax41.set_xlim(vsh_min,vsh_max)
ax41.xaxis.label.set_color("g")
ax41.tick_params(axis='x', colors="g")
ax41.spines["top"].set_position(("axes", 1.07))
ax41.spines["top"].set_visible(True)
ax41.spines["top"].set_edgecolor("g")
ax41.set_xticks(np.arange(vsh_max, vsh_min+.01, ((vsh_min-vsh_max)/(ax4_ticks-1))))
ax41.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### VSH SP ###


ax42.plot(df[crv_vsh_sp], df.index, color = "red", linewidth = 1)
ax42.set_xlabel(f"VSH ({crv_vsh_sp})")
ax42.set_xlim(vsh_min,vsh_max)
ax42.xaxis.label.set_color("r")
ax42.tick_params(axis='x', colors="r")
ax42.spines["top"].set_position(("axes", 1.12))
ax42.spines["top"].set_visible(True)
ax42.spines["top"].set_edgecolor("r")
ax42.set_xticks(np.arange(vsh_max, vsh_min+.01, ((vsh_min-vsh_max)/(ax4_ticks-1))))
ax42.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))


fig.suptitle(las.well.WELL.value+"\n"+"AREA: "+las.well.FLD.value+"\n"+"COMPANY: "+las.well.COMP.value,size=26,
             x=0.36,y=1)


for ax in [ax1, ax2, ax3, ax4]:
    ax.set_ylim(bot_plot, top_plot)
    ax.grid(which='major', color='lightgrey', linestyle='-')
    ax.xaxis.set_ticks_position("top")
    ax.xaxis.set_label_position("top")
    ax.spines["top"].set_position(("axes", 1.02))
 
    
for ax in [ax2, ax3, ax4]:
    plt.setp(ax.get_yticklabels(), visible = False)
   

plt.tight_layout()
fig.subplots_adjust(wspace = 0.2)

mplcursors.cursor(hover=True)

plt.show()


# [>> RETURN TO VSH ESTIMATION CELL  << (please rename well_xxxx_xxx FILE for processing the next zone)](#VSH)

<a id='df_final'></a>
# <font color='#D35400'>FINAL DATAFRAME CREATION</font>

- this cell merges all *well_xxxx_xxxx_final* files created in one dataframe

In [ ]:
print('''\033[91m\033[1m¡¡YOU ARE ABOUT TO CREATE THE FINAL DATA FILE!!

IF YOU HAVE ALREADY PROCESSED ALL ZONES PRESS ANY KEY TO CONTINUE.\n
IF NOT, PLEASE NOT CONTINUE (STOP CELL)''')
wait = input("")
clear_output(wait=True)



well_list = []
for match in fnmatch.filter(dir(), 'well_*_final'):
    if match == 'well_zones' or match=='well_nan':
        pass
    else:    
        well_list.append(match)
print(f'DATASETS FINALIZADOS: {well_list}')
        

well_final = pd.DataFrame()
well_work_name = 'DataFrame Final: well_final'
for a,b in enumerate(well_list):                                                                                                           
    well_final = well_final.append(globals()[f'{well_list[a]}'],ignore_index=False)
print ('\033[1m'+'\n'+well_work_name)    
well_final

<a id='combo_final'></a>
# <font color='#D35400'>FINAL DATAFRAME DATA VISUALIZATION</font>

In [ ]:
df = well_final

#####################################################

# TOP & BOTTOM 

top_plot=well_zone_file.index[0]
bot_plot=well_zone_file.index[-1]



# NUMBER OF TICKS IN THE TRACK

ax1_ticks = 6         # Convencionales 
ax2_ticks = 6         # Resistividades
ax3_ticks = 7         # Porosidades
ax4_ticks = 5         # VSH

########################################################

#  VSH CURVES

crv_vsh_dn = 'vsh_dn'
crv_vsh_gr = 'vsh_gr' 
crv_vsh_sp = 'vsh_sp'

if crv_vsh_dn in df.columns:
    pass
else:
    df[crv_vsh_dn]=np.nan
    
    
if crv_vsh_gr in df.columns:
    pass
else:
    df[crv_vsh_gr]=np.nan
    
    
if crv_vsh_sp in df.columns:
    pass
else:
    df[crv_vsh_sp]=np.nan


############################

fig, ax = plt.subplots(figsize=(20,20))#Set up the plot axes
   
ax1 = plt.subplot2grid((1,6), (0,0), rowspan=1, colspan = 1) 
ax11 = ax1.twiny()
ax12 = ax1.twiny()

ax2 = plt.subplot2grid((1,6), (0,1), rowspan=1, colspan = 1, sharey = ax1)
ax21 = ax2.twiny()

ax3 = plt.subplot2grid((1,6), (0,2), rowspan=1, colspan = 1, sharey = ax1) #
ax31 = ax3.twiny() 
ax32 = ax3.twiny() 

ax4 = plt.subplot2grid((1,6), (0,3), rowspan=1, colspan = 1, sharey = ax1)
ax41 = ax4.twiny() 
ax42 = ax4.twiny()


################################################################################

### CAL ###
cal_min = 6
cal_max = 16

ax1.plot(df[crv_cal], df.index, color = "black", linewidth = 0.5)
ax1.set_ylabel("Depth (m)")

if abs(top_plot-bot_plot) > 500:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/99)))
    
elif abs(top_plot-bot_plot) <= 500 and abs(top_plot-bot_plot) > 200:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/49)))
else:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/24)))
    
ax1.set_xlabel(f"Caliper {df[crv_cal].name}")
ax1.set_xlim(cal_min, cal_max)
ax1.xaxis.label.set_color("black")
ax1.tick_params(axis='x', colors="black")
ax1.spines["top"].set_edgecolor("black")
ax1.set_xticks(np.arange(cal_min, cal_max+1, ((cal_max-cal_min)/(ax1_ticks-1))))
ax1.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))

ax1.fill_betweenx(df.index, df[crv_bit], df[crv_cal], where=df[crv_bit]>=df[crv_cal], interpolate=True, color='brown')
ax1.fill_betweenx(df.index, df[crv_bit], df[crv_cal], where=df[crv_bit]<=df[crv_cal], interpolate=True, color='#a6ecfa')
################################################################################

### GR ###
gr_min=0
gr_max=200

ax11.plot(df[crv_gr], df.index, color = "green", linewidth = 1.5)
ax11.set_xlabel(f"Gamma Ray ({df[crv_gr].name})")
ax11.xaxis.label.set_color("green")
ax11.set_xlim(gr_min, gr_max)
ax11.tick_params(axis='x', colors="green")
ax11.spines["top"].set_position(("axes", 1.12))
ax11.spines["top"].set_visible(True)
ax11.spines["top"].set_edgecolor("green")
ax11.title.set_color('green')
ax11.set_xticks(np.arange(gr_min, gr_max+1, ((gr_max-gr_min)/(ax1_ticks-1))))
ax11.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################

### SP ###
sp_min=-80
sp_max=20

ax12.plot(df[crv_sp], df.index, color = "red", linewidth = 1.5)
ax12.set_xlabel(f"SP ({df[crv_sp].name})")
ax12.xaxis.label.set_color("red")
ax12.set_xlim(sp_min,sp_max)
ax12.tick_params(axis='x', colors="r")
ax12.spines["top"].set_position(("axes", 1.07))
ax12.spines["top"].set_visible(True)
ax12.spines["top"].set_edgecolor("r")
ax12.title.set_color('r')
ax12.set_xticks(np.arange(sp_min, sp_max+1, ((sp_max-sp_min)/(ax1_ticks-1))))
ax12.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################

### RESD ###
res_min=0.1
res_max=100

ax2.plot(df[crv_dp_res], df.index, color = "blue", linewidth = 2)
ax2.set_xlabel(f"Deep Resistivity ({df[crv_dp_res].name})")
ax2.set_xlim(res_min, res_max)
ax2.xaxis.label.set_color("blue")
ax2.tick_params(axis='x', colors="blue")
ax2.spines["top"].set_edgecolor("blue")
ax2.set_xticks(np.arange(res_min, res_max+.1, (res_max/(ax2_ticks-1))))
ax2.set_xscale("log")
ax2.grid(True, which="both", axis='x')
ax2.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
################################################################################

### RESS ###


ax21.plot(df[crv_sh_res], df.index, color = "r", linewidth = 0.5)
ax21.set_xlabel(f"Shallow Resistivity ({df[crv_sh_res].name})")
ax21.set_xlim(res_min, res_max)
ax21.xaxis.label.set_color("r")
ax21.spines["top"].set_position(("axes", 1.07))
ax21.spines["top"].set_visible(True)
ax21.tick_params(axis='x', colors="r")
ax21.spines["top"].set_edgecolor("r")
ax21.set_xticks(np.arange(res_min, res_max+.1, ((res_max-res_min)/(ax2_ticks-1))))
ax21.set_xscale("log")
ax21.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
################################################################################

### DEN BULK ###
denbulk_min = 2.65
denbulk_max = 1.65


ax3.plot(df[crv_den_bulk], df.index, color = "red", linewidth = 1.0)
ax3.set_xlabel(f"Density Bulk ({df[crv_den_bulk].name})")
ax3.set_xlim(denbulk_min,denbulk_max)
ax3.xaxis.label.set_color("red")
ax3.tick_params(axis='x', colors="r")
ax3.spines["top"].set_edgecolor("r")
ax3.set_xticks(np.arange(denbulk_max, denbulk_min+.01, ((denbulk_min-denbulk_max)/(ax3_ticks-1))))
ax3.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### NEU POR ###
neupor_min = .6
neupor_max = 0

ax31.plot(df[crv_neu_por], df.index, color = "blue", linewidth = 1.0)
ax31.set_xlabel(f"Neutron Porosity ({df[crv_neu_por].name})")
ax31.xaxis.label.set_color("b")
ax31.set_xlim(neupor_min,neupor_max)
ax31.tick_params(axis='x', colors="blue")
ax31.spines["top"].set_position(("axes", 1.07))
ax31.spines["top"].set_visible(True)
ax31.spines["top"].set_edgecolor("blue")
ax31.set_xticks(np.arange(neupor_max, neupor_min+.01, (abs(neupor_max-neupor_min)/(ax3_ticks-1))))
ax31.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### NMR POR ###
nmrtpor_min = .6
nmrtpor_max = 0

ax32.plot(df[crv_nmr_por], df.index, color = "black", linewidth = 1.0)
ax32.set_xlabel(f"NMR Total Porosity ({df[crv_nmr_por].name})")
ax32.xaxis.label.set_color("black")
ax32.set_xlim(nmrtpor_min,nmrtpor_max)
ax32.tick_params(axis='x', colors="black")
ax32.spines["top"].set_position(("axes", 1.12))
ax32.spines["top"].set_visible(True)
ax32.spines["top"].set_edgecolor("black")
ax32.set_xticks(np.arange(nmrtpor_max, nmrtpor_min+.01, (abs(nmrtpor_max-nmrtpor_min)/(ax3_ticks-1))))
ax32.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### VSH DEN-NEU ###
vsh_min = 1
vsh_max = 0

ax4.plot(df[crv_vsh_dn], df.index, color = "black", linewidth = 1)
ax4.set_xlabel(f"VSH ({crv_vsh_dn})")
ax4.set_xlim(vsh_min,vsh_max)
ax4.xaxis.label.set_color("black")
ax4.tick_params(axis='x', colors="black")
ax4.spines["top"].set_edgecolor("black")
ax4.set_xticks(np.arange(vsh_max, vsh_min+.01, ((vsh_min-vsh_max)/(ax4_ticks-1))))
ax4.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### VSH GR ###


ax41.plot(df[crv_vsh_gr], df.index, color = "green", linewidth = 1)
ax41.set_xlabel(f"VSH ({crv_vsh_gr})")
ax41.set_xlim(vsh_min,vsh_max)
ax41.xaxis.label.set_color("g")
ax41.tick_params(axis='x', colors="g")
ax41.spines["top"].set_position(("axes", 1.07))
ax41.spines["top"].set_visible(True)
ax41.spines["top"].set_edgecolor("g")
ax41.set_xticks(np.arange(vsh_max, vsh_min+.01, ((vsh_min-vsh_max)/(ax4_ticks-1))))
ax41.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### VSH SP ###


ax42.plot(df[crv_vsh_sp], df.index, color = "red", linewidth = 1)
ax42.set_xlabel(f"VSH ({crv_vsh_sp})")
ax42.set_xlim(vsh_min,vsh_max)
ax42.xaxis.label.set_color("r")
ax42.tick_params(axis='x', colors="r")
ax42.spines["top"].set_position(("axes", 1.12))
ax42.spines["top"].set_visible(True)
ax42.spines["top"].set_edgecolor("r")
ax42.set_xticks(np.arange(vsh_max, vsh_min+.01, ((vsh_min-vsh_max)/(ax4_ticks-1))))
ax42.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))



fig.suptitle(las.well.WELL.value+"\n"+"AREA: "+las.well.FLD.value+"\n"+"COMPANY: "+las.well.COMP.value,size=26,
             x=0.36,y=1)

# Common functions for setting up the plot can be extracted into
# a for loop. This saves repeating code.

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_ylim(bot_plot, top_plot)
    ax.grid(which='major', color='lightgrey', linestyle='-')
    ax.xaxis.set_ticks_position("top")
    ax.xaxis.set_label_position("top")
    ax.spines["top"].set_position(("axes", 1.02))
 
    
for ax in [ax2, ax3, ax4]:
    plt.setp(ax.get_yticklabels(), visible = False)
   

plt.tight_layout()
fig.subplots_adjust(wspace = 0.2)

mplcursors.cursor(hover=True)

plt.show()